In [1]:
from typing import List
from PIL.Image import Image as PILImage
from PIL import ImageDraw, ImageFont
import random
import pytesseract
from PIL import Image, ImageOps


In [2]:
class Letter:
    def __init__(self, letter: str, index: int) -> None:
        self.letter = letter
        self.index = index
        self._in_correct_place = False
        self._in_word = False

    def is_in_correct_place(self) -> bool:
        return self._in_correct_place
    
    def set_in_correct_place(self, value: bool) -> None:
        self._in_correct_place = value
    
    def is_in_word(self) -> bool:
        return self._in_word
    
    def set_in_word(self, value: bool) -> None:
        self._in_word = value

In [3]:
class Guess:
    def __init__(self, word: str) -> None:
        self.word: str = word.upper()
        self.letters: list[Letter] = []
        
    def get_word(self) -> str:
        return self.word
    
    def get_guesses(self) -> list[Letter]:
        return self.letters
    
    def add_guess(self, guess: Letter) -> None:
        self.letters.append(guess)

    def is_correct_position(self, index: int) -> bool:
        return self.letters[index].is_in_correct_place()

    def is_in_word(self, index: int) -> bool:
        return self.letters[index].is_in_word()

    def is_correct_guess(self) -> bool:
        return all(letter.is_in_correct_place() for letter in self.letters)


In [11]:
class GameEngine:
    def __init__(self, target_word: str) -> None:
        self.target_word = target_word.upper()
        
    def evaluate_guess(self, guess: Guess) -> list[Letter]:
        guess = guess.get_word().upper()
        
        letters=[]
        for i in range(len(guess)):
            letter=Letter(guess[i], i)
            
            if guess[i] == self.target_word[i]:
                letter.set_in_correct_place(True)
            
            if guess[i] in self.target_word:
                letter.set_in_word(True)
            
            letters.append(letter)

        return letters


class Bot:
    def __init__(self, target_word: str) -> None:
        self.target_word = target_word.upper()

    def play(self, num_max_guesses: int, word_list: List[str], game_master: GameEngine) -> None:
        max_guesses = num_max_guesses
        guess_count =  0
        guesses: List[str] = []
        feedback_images = []

        while guess_count < max_guesses:
            guess = self._create_guess(word_list)
            guesses.append(guess)
            guess_count += 1
            guess_obj = Guess(guess)
            evaluated_letters = game_master.evaluate_guess(guess_obj)
            guess_obj.letters = evaluated_letters
            feedback_img = self._format_guess_feedback(guess_obj)
            feedback_images.append(feedback_img)
            if guess_obj.is_correct_guess():
                print(f"yes! The word is {guess}!")
                break
        else:
            print(f"you ran out of guesses. The word was {game_master.target_word}.")

        final_image = self._combine_feedback_images(feedback_images)
        final_image.show()

    def _create_guess(self, word_list: List[str]) -> str:
        return random.choice(word_list)
    
    def _format_guess_feedback(self, guess_obj: Guess) -> Image:
        return self.make_visual_guess(guess_obj)
    
    
    def _combine_feedback_images(self, feedback_images: List[PILImage]) -> Image:
        total_height = sum([img.height for img in feedback_images])
        max_width = max([img.width for img in feedback_images])

        combined_image = Image.new("RGBA", (max_width, total_height), color="white")

        y_offset = 0
        for img in feedback_images:
            combined_image.paste(img, (0, y_offset))
            y_offset += img.height

        return combined_image

    def make_visual_guess(self, guess: Guess) -> Image:
        def render_word(guess: str, correct_word: str):
            letters=[]
            for loc, letter in enumerate(guess):
                if correct_word[loc]==letter:
                    letters.append(render_letter(letter, in_word=True, correct_place=True))
                elif letter in correct_word:
                    letters.append(render_letter(letter, in_word=True))
                else:
                    letters.append(render_letter(letter))


            space_between_letters=5
            word_width=(len(letters)*letters[0].width)+(len(letters)-1)*space_between_letters
            word_height=letters[0].height

            wrd=Image.new('RGBA', (word_width,word_height), color="white")
            curr_loc=0
            for char in letters:
                wrd.paste(char,(curr_loc,0))
                curr_loc+=char.width+space_between_letters
            return wrd


        def render_letter(letter:str, in_word:bool=False, correct_place:bool=False)->PILImage:
            block_width=80
            block_height=80
            x=block_width//2
            y=block_height//2

            color:str="darkgrey"
            if correct_place:
                color="green"
            elif in_word:
                color="yellow"

            blk=Image.new('RGBA', (block_width, block_height), color=color)

            draw = ImageDraw.Draw(blk)

            # use a bitmap font
            font_size=50
            font = ImageFont.truetype("course5/assets/roboto_font/Roboto-Bold.ttf", font_size)

            # see https://pillow.readthedocs.io/en/stable/handbook/text-anchors.html#text-anchors
            draw.text((x, y), letter, size=50, anchor="mm",  font=font)
            return blk

        return render_word(guess.get_word(), self.target_word)




In [12]:
if __name__ == "__main__":
    word_list = ["dream", "radio", "peace", "snack", "noise"]
    game_engine = GameEngine("radio")
    bot = Bot(game_engine.target_word)
    # bot = Bot()
    bot.play(6, word_list, game_engine)

yes! The word is radio!
